In [2]:
import numpy as np
import random
from pathlib import Path
import progressbar as pb
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import pandas as pd


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Carregar os datasets disponibilizados pela DNC ou fazer download via
Kaggle dos datasets

In [4]:
# Load the data 
books_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ARQUIVOS /Books.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ARQUIVOS /Ratings.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ARQUIVOS /Users.csv')

In [5]:
books_df.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [6]:
ratings_df.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [7]:
users_df.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.000
2,3,"moscow, yukon territory, russia",NaN


2. Implementar o pipeline de transformação de dados para obter a
contagem dos clientes que mais avaliam livros.


In [8]:
n_unique_users = users_df["User-ID"].nunique()
n_unique_users

278858

3 - Implementar o pipeline de transformação de dados para obter a contagem dos livros mais avaliados.

In [9]:
n_unique_itens = books_df['ISBN'].nunique()
n_unique_itens

271360

In [10]:

def start_pipeline(dataf):
    return dataf.copy()

def get_user_counts(dataf):
  return dataf.groupby(by='User-ID', as_index=False) \
    .agg({'ISBN': pd.Series.nunique}) \
    .rename(columns={'ISBN': 'ISBN_count'}) \
    .set_index('User-ID') \
    .sort_values('ISBN_count', ascending = False)

def get_book_counts(dataf):
  return dataf.groupby(by='ISBN', as_index=False) \
    .agg({'User-ID': pd.Series.nunique}) \
    .rename(columns={'User-ID': 'User-ID_count'}) \
    .set_index('ISBN') \
    .sort_values('User-ID_count', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(sample_size) \
    .index \
    .tolist()

sample_size = 1000
top_user_ids = ratings_df \
  .pipe(start_pipeline) \
  .pipe(get_user_counts) \
  .pipe(get_n_top_values, sample_size)

sample_size = 999
top_book_ids = ratings_df \
  .pipe(start_pipeline) \
  .pipe(get_book_counts) \
  .pipe(get_n_top_values, sample_size)

In [11]:
len(top_user_ids), len(top_book_ids)

(1000, 999)

In [12]:
def transform_sample(dataf):
  return pd.DataFrame({
      "user": dataf["User-ID"],
      "book": dataf["ISBN"],
      "action": dataf["Book-Rating"]
  })

matriz_de_relacionamento_user_sampled = ratings_df[ratings_df["User-ID"].isin(top_user_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="book", values="action").fillna(0)

In [13]:
matriz_de_relacionamento_user_sampled.shape

(1000, 212171)

In [14]:
matriz_de_relacionamento_book_sampled = ratings_df[ratings_df["ISBN"].isin(top_book_ids)] \
  .pipe(transform_sample) \
  .pivot_table(index="user", columns="book", values="action").fillna(0)

In [15]:
cos_user = cosine_similarity(matriz_de_relacionamento_user_sampled)

print(cos_user)

[[1.         0.0528255  0.         ... 0.         0.         0.        ]
 [0.0528255  1.         0.04675324 ... 0.         0.         0.        ]
 [0.         0.04675324 1.         ... 0.02334216 0.         0.        ]
 ...
 [0.         0.         0.02334216 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [16]:
cos_item = cosine_similarity(matriz_de_relacionamento_book_sampled.T)

print(cos_item)
cos_item.shape

[[1.         0.05983705 0.02857749 ... 0.02202635 0.01551346 0.        ]
 [0.05983705 1.         0.01555774 ... 0.02620312 0.         0.        ]
 [0.02857749 0.01555774 1.         ... 0.04822782 0.         0.        ]
 ...
 [0.02202635 0.02620312 0.04822782 ... 1.         0.         0.        ]
 [0.01551346 0.         0.         ... 0.         1.         0.03301185]
 [0.         0.         0.         ... 0.         0.03301185 1.        ]]


(999, 999)

In [17]:
print(cos_user)

[[1.         0.0528255  0.         ... 0.         0.         0.        ]
 [0.0528255  1.         0.04675324 ... 0.         0.         0.        ]
 [0.         0.04675324 1.         ... 0.02334216 0.         0.        ]
 ...
 [0.         0.         0.02334216 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [18]:
matriz_de_relacionamento_user_sampled.index

Int64Index([   254,   2033,   2276,   2766,   2977,   3363,   3757,   4017,
              4385,   6242,
            ...
            274301, 274308, 274808, 275970, 276680, 277427, 277478, 277639,
            278188, 278418],
           dtype='int64', name='user', length=1000)

In [26]:
cliente_de_interesse = matriz_de_relacionamento_user_sampled.index[random.randint(0, matriz_de_relacionamento_user_sampled.shape[0])]

print(f"Cliente de interesse: {cliente_de_interesse}")

cliente_idx = matriz_de_relacionamento_user_sampled.index.tolist().index(cliente_de_interesse)
print(f"Cliente id: {cliente_de_interesse},tem índice {cliente_idx}")

closest_10_users = np.argsort(-cos_user[cliente_idx])[:10]

for i in zip(matriz_de_relacionamento_user_sampled.index[closest_10_users], cos_user[cliente_idx][closest_10_users]):
    print(f"cliente {i[0]} tem similaridade {i[1]:.2f} com cliente {cliente_de_interesse}")

Cliente de interesse: 27647
Cliente id: 27647,tem índice 82
cliente 27647 tem similaridade 1.00 com cliente 27647
cliente 251844 tem similaridade 0.08 com cliente 27647
cliente 26883 tem similaridade 0.06 com cliente 27647
cliente 13082 tem similaridade 0.06 com cliente 27647
cliente 80538 tem similaridade 0.06 com cliente 27647
cliente 76223 tem similaridade 0.06 com cliente 27647
cliente 264317 tem similaridade 0.05 com cliente 27647
cliente 69971 tem similaridade 0.05 com cliente 27647
cliente 221445 tem similaridade 0.04 com cliente 27647
cliente 106007 tem similaridade 0.04 com cliente 27647


In [20]:
matriz_de_relacionamento_book_sampled.T.index

Index(['000649840X', '002542730X', '0060008032', '0060096195', '006016848X',
       '0060173289', '0060175400', '0060188731', '006019491X', '0060199652',
       ...
       '1573225517', '1573225789', '1573227331', '1573229326', '1573229571',
       '1573229725', '1576737330', '1592400876', '1878424319', '8873122933'],
      dtype='object', name='book', length=999)

In [21]:
print(cos_item)

[[1.         0.05983705 0.02857749 ... 0.02202635 0.01551346 0.        ]
 [0.05983705 1.         0.01555774 ... 0.02620312 0.         0.        ]
 [0.02857749 0.01555774 1.         ... 0.04822782 0.         0.        ]
 ...
 [0.02202635 0.02620312 0.04822782 ... 1.         0.         0.        ]
 [0.01551346 0.         0.         ... 0.         1.         0.03301185]
 [0.         0.         0.         ... 0.         0.03301185 1.        ]]


In [22]:
def mostra_n_mais_proximos(id_de_interesse, lista_instanciast, matriz_similaridade, n):
  print(f"Item de interesse: {instancia_de_interesse}")

  assert id_de_interesse in lista_instanciast
  cliente_idx = lista_instanciast.tolist().index(instancia_de_interesse)
  print(f"Item id: {instancia_de_interesse}, tem índice {cliente_idx}")

  closest_10_users = np.argsort(-matriz_similaridade[cliente_idx])[:n]
  print(instancia_de_interesse, closest_10_users)

  for i in zip(lista_instanciast[closest_10_users], matriz_similaridade[cliente_idx][closest_10_users]):
      print(f"Item {i[0]} tem similaridade {i[1]:.2f} com item {instancia_de_interesse}")


lista_instancias = matriz_de_relacionamento_book_sampled.T.index
instancia_de_interesse = lista_instancias[random.randint(0, len(lista_instancias))]

mostra_n_mais_proximos(instancia_de_interesse, lista_instancias, cos_item, 10)

Item de interesse: 0425144429
Item id: 0425144429, tem índice 392
0425144429 [392 196 197 200 403 389 637 396 623 919]
Item 0425144429 tem similaridade 1.00 com item 0425144429
Item 0345384369 tem similaridade 0.11 com item 0425144429
Item 0345384377 tem similaridade 0.11 com item 0425144429
Item 0345386108 tem similaridade 0.11 com item 0425144429
Item 0425153975 tem similaridade 0.11 com item 0425144429
Item 0425140032 tem similaridade 0.11 com item 0425144429
Item 0451181379 tem similaridade 0.09 com item 0425144429
Item 0425147517 tem similaridade 0.08 com item 0425144429
Item 0451156609 tem similaridade 0.08 com item 0425144429
Item 0743467523 tem similaridade 0.08 com item 0425144429


In [23]:
cor_users = np.corrcoef(matriz_de_relacionamento_user_sampled)
print(cor_users)
cor_users.shape

[[ 1.00000000e+00  5.24499768e-02 -5.03368731e-04 ... -1.91344402e-04
  -2.48344082e-04 -3.66792106e-04]
 [ 5.24499768e-02  1.00000000e+00  4.60295960e-02 ... -2.88958574e-04
  -3.75036588e-04 -5.53910764e-04]
 [-5.03368731e-04  4.60295960e-02  1.00000000e+00 ...  2.29898773e-02
  -4.73873967e-04 -6.99888756e-04]
 ...
 [-1.91344402e-04 -2.88958574e-04  2.29898773e-02 ...  1.00000000e+00
  -1.80132625e-04 -2.66047109e-04]
 [-2.48344082e-04 -3.75036588e-04 -4.73873967e-04 ... -1.80132625e-04
   1.00000000e+00 -3.45300015e-04]
 [-3.66792106e-04 -5.53910764e-04 -6.99888756e-04 ... -2.66047109e-04
  -3.45300015e-04  1.00000000e+00]]


(1000, 1000)

In [25]:
cor_books = np.corrcoef(matriz_de_relacionamento_book_sampled.T)
print(cor_books)
cor_books.shape

[[ 1.00000000e+00  5.84285607e-02  2.75927524e-02 ...  2.07534252e-02
   1.42178559e-02 -8.30758829e-04]
 [ 5.84285607e-02  1.00000000e+00  1.41896293e-02 ...  2.44633695e-02
  -1.80673898e-03 -1.14023195e-03]
 [ 2.75927524e-02  1.41896293e-02  1.00000000e+00 ...  4.70823544e-02
  -1.21870506e-03 -7.69124075e-04]
 ...
 [ 2.07534252e-02  2.44633695e-02  4.70823544e-02 ...  1.00000000e+00
  -1.56520034e-03 -9.87797045e-04]
 [ 1.42178559e-02 -1.80673898e-03 -1.21870506e-03 ... -1.56520034e-03
   1.00000000e+00  3.20496992e-02]
 [-8.30758829e-04 -1.14023195e-03 -7.69124075e-04 ... -9.87797045e-04
   3.20496992e-02  1.00000000e+00]]


(999, 999)